# Part I: Understanding the Problem and Data Exploration
<img src="IMG/workflow.png">

## Frame the problem

Before looking at the data it is important to understand how does the company expect to use and benefit from this model? This first brainstorming helps to determine how to frame the problem, what algorithms to select and measure the performance of each one.

## Make Assumptions

It is good to make some assumptions on what possible outcomes we might expect from our analysis according to the available data. Therefore, by knowing the goal we should think which possible factors might affect the sales prediction outcome.

# Start Data Exploration (EDA)
* browse and clean data
* look univariate and multivariate statistics
* isolate and investigate the statistics of groups
* visualize interesting findings
* prepare a small report on your findings (in a seperate notebook)

## Getting Started


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings # Ignores any warning
warnings.filterwarnings("ignore")

#Important. for now, we only look at the available train data
train = pd.read_csv("DATA/Train.csv") 


In [2]:
train.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [18]:
train.isnull().any()


Item_Identifier              False
Item_Weight                   True
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                   True
Outlet_Location_Type         False
Outlet_Type                  False
Item_Outlet_Sales            False
dtype: bool

In [19]:
train_nans = train.isnull()
train_nans = train_nans.any(axis=1)
train_nans


0       False
1       False
2       False
3        True
4       False
5       False
6       False
7        True
8        True
9        True
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18       True
19      False
20      False
21       True
22      False
23       True
24      False
25       True
26      False
27      False
28       True
29       True
        ...  
8493     True
8494     True
8495    False
8496     True
8497    False
8498    False
8499    False
8500     True
8501     True
8502     True
8503    False
8504     True
8505    False
8506    False
8507    False
8508     True
8509     True
8510    False
8511    False
8512    False
8513    False
8514     True
8515    False
8516    False
8517    False
8518    False
8519     True
8520    False
8521    False
8522    False
Length: 8523, dtype: bool

In [20]:
# Erkenntnis: Es gibt nur nans in weigth und size --> nummerische Attribute --> mathematische Ansätze (median/mean) zur Ersetzung von nans möglich
# Idee: Korrelation zwischen weight und type rauszufinden und dann ggf. 
train.corr()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
Item_Weight,1.000000,-0.014048,0.027141,-0.011588,0.014123
Item_Visibility,-0.014048,1.000000,-0.001315,-0.074834,-0.128625
Item_MRP,0.027141,-0.001315,1.000000,0.005020,0.567574
Outlet_Establishment_Year,-0.011588,-0.074834,0.005020,1.000000,-0.049135
Item_Outlet_Sales,0.014123,-0.128625,0.567574,-0.049135,1.000000


In [21]:
def fat_to_numeric(x):
    if (x == 'Low Fat') or (x == 'low fat') or (x == 'LF'):
        return 0
    if (x == 'Regular') or (x == 'reg'):
        return 1
    
def type_to_numeric(x):
    if (x == 'Dairy'):
        return 0
    if (x == 'Soft Drinks'):
        return 1
    if (x == 'Meat'):
        return 2
    if (x == 'Fruits and Vegetables'):
        return 3
    if (x == 'Household'):
        return 4
    if (x == 'Baking Goods'):
        return 5
    if (x == 'Snack Foods'):
        return 6
    if (x == 'Frozen Foods'):
        return 7
    if (x == 'Breakfast'):
        return 8
    if (x == 'Health and Hygiene'):
        return 9
    if (x == 'Hard Drinks'):
        return 10
    if (x == 'Canned'):
        return 11
    if (x == 'Breads'):
        return 12
    if (x == 'Starchy Foods'):
        return 13
    if (x == 'Others'):
        return 14
    if (x == 'Seafood'):
        return 15
    
def size_to_numeric(x):
    if (x == 'Small'):
        return 0
    if (x == 'Medium'):
        return 1
    if (x == 'High'):
        return 2
    
def loc_type_to_numeric Tier 1  

Supermarket Type1

In [25]:
train['Item_Type_n'] = train['Item_Type'].apply(type_to_numeric)

In [27]:
train.corr()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Item_Type_n
Item_Weight,1.000000,-0.014048,0.027141,-0.011588,0.014123,-0.032719
Item_Visibility,-0.014048,1.000000,-0.001315,-0.074834,-0.128625,-0.015682
Item_MRP,0.027141,-0.001315,1.000000,0.005020,0.567574,-0.028493
Outlet_Establishment_Year,-0.011588,-0.074834,0.005020,1.000000,-0.049135,0.011040
Item_Outlet_Sales,0.014123,-0.128625,0.567574,-0.049135,1.000000,-0.007849
Item_Type_n,-0.032719,-0.015682,-0.028493,0.011040,-0.007849,1.000000


In [28]:
train['Item_Fat_Content_n'] = train['Item_Fat_Content'].apply(fat_to_numeric)

In [29]:
train.corr()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Item_Type_n,Item_Fat_Content_n
Item_Weight,1.000000,-0.014048,0.027141,-0.011588,0.014123,-0.032719,-0.023238
Item_Visibility,-0.014048,1.000000,-0.001315,-0.074834,-0.128625,-0.015682,0.047314
Item_MRP,0.027141,-0.001315,1.000000,0.005020,0.567574,-0.028493,0.006063
Outlet_Establishment_Year,-0.011588,-0.074834,0.005020,1.000000,-0.049135,0.011040,0.003151
Item_Outlet_Sales,0.014123,-0.128625,0.567574,-0.049135,1.000000,-0.007849,0.018719
Item_Type_n,-0.032719,-0.015682,-0.028493,0.011040,-0.007849,1.000000,-0.038825
Item_Fat_Content_n,-0.023238,0.047314,0.006063,0.003151,0.018719,-0.038825,1.000000


In [30]:
types = train.groupby('Item_Type')# ["Item_Wight"]
for grp, data in types:
    print(grp+ ':')
    print('mean: ' + str(data['Item_Weight'].mean()))
    print('median: ' + str(data['Item_Weight'].median()))
    print( ' ')

Baking Goods:
mean: 12.277108208955255
median: 11.65
 
Breads:
mean: 11.3469362745098
median: 10.6
 
Breakfast:
mean: 12.768202247191002
median: 10.695
 
Canned:
mean: 12.305705009276451
median: 12.15
 
Dairy:
mean: 13.42606890459367
median: 13.35
 
Frozen Foods:
mean: 12.867061281337076
median: 12.85
 
Fruits and Vegetables:
mean: 13.224769381746881
median: 13.1
 
Hard Drinks:
mean: 11.400327868852452
median: 10.1
 
Health and Hygiene:
mean: 13.142313953488392
median: 12.15
 
Household:
mean: 13.384736495388726
median: 13.15
 
Meat:
mean: 12.81734421364986
median: 12.35
 
Others:
mean: 13.853284671532842
median: 14.5
 
Seafood:
mean: 12.5528431372549
median: 11.65
 
Snack Foods:
mean: 12.987879554655919
median: 13.15
 
Soft Drinks:
mean: 11.847459893048129
median: 11.8
 
Starchy Foods:
mean: 13.690730769230765
median: 13.175
 


In [ ]:
# --> meadian und mean liegen relativ nahe beisammen und zwar bei allen Typen --> keine einseitigen Ausreiser --> für nans in weght median/ mean benutzen

In [36]:
train['Item_Weight'].fillna(train['Item_Weight'].mean() ,inplace=True)

In [ ]:
# gleiches noch für outletsize
types = train.groupby('Item_Type')# ["Item_Wight"]
for grp, data in types:
    print(grp+ ':')
    print('mean: ' + str(data['Item_Weight'].mean()))
    print('median: ' + str(data['Item_Weight'].median()))
    print( ' ')

Item_Identifier              False
Item_Weight                  False
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                   True
Outlet_Location_Type         False
Outlet_Type                  False
Item_Outlet_Sales            False
Item_Type_n                  False
Item_Fat_Content_n           False
dtype: bool